# Exploring the Dataset

In [1]:
from flatten_json import flatten
import pandas as pd
import seaborn as sns

import os

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("FraudTransactionModel") \
    .getOrCreate()

# Read JSON file into dataframe    
df = spark.read.json("Data/transactions.txt")
df.printSchema()
df.show()


root
 |-- accountNumber: string (nullable = true)
 |-- accountOpenDate: string (nullable = true)
 |-- acqCountry: string (nullable = true)
 |-- availableMoney: double (nullable = true)
 |-- cardCVV: string (nullable = true)
 |-- cardLast4Digits: string (nullable = true)
 |-- cardPresent: boolean (nullable = true)
 |-- creditLimit: double (nullable = true)
 |-- currentBalance: double (nullable = true)
 |-- currentExpDate: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- dateOfLastAddressChange: string (nullable = true)
 |-- echoBuffer: string (nullable = true)
 |-- enteredCVV: string (nullable = true)
 |-- expirationDateKeyInMatch: boolean (nullable = true)
 |-- isFraud: boolean (nullable = true)
 |-- merchantCategoryCode: string (nullable = true)
 |-- merchantCity: string (nullable = true)
 |-- merchantCountryCode: string (nullable = true)
 |-- merchantName: string (nullable = true)
 |-- merchantState: string (nullable = true)
 |-- merchantZip: string (nullable =

In [3]:
pandaDf = df.toPandas()

In [4]:
pandaDf.head()

,accountNumber,accountOpenDate,acqCountry,availableMoney,cardCVV,cardLast4Digits,cardPresent,creditLimit,currentBalance,currentExpDate,...,merchantName,merchantState,merchantZip,posConditionCode,posEntryMode,posOnPremises,recurringAuthInd,transactionAmount,transactionDateTime,transactionType
0,737265056,2015-03-14,US,5000.0,414,1803,False,5000.0,0.0,06/2023,...,Uber,,,01,02,,,98.55,2016-08-13T14:27:32,PURCHASE
1,737265056,2015-03-14,US,5000.0,486,767,True,5000.0,0.0,02/2024,...,AMC #191138,,,01,09,,,74.51,2016-10-11T05:05:54,PURCHASE
2,737265056,2015-03-14,US,5000.0,486,767,False,5000.0,0.0,08/2025,...,Play Store,,,01,09,,,7.47,2016-11-08T09:18:39,PURCHASE
3,737265056,2015-03-14,US,5000.0,486,767,False,5000.0,0.0,08/2025,...,Play Store,,,01,09,,,7.47,2016-12-10T02:14:50,PURCHASE
4,830329091,2015-08-06,US,5000.0,885,3143,True,5000.0,0.0,10/2029,...,Tim Hortons #947751,,,01,02,,,71.18,2016-03-24T21:04:46,PURCHASE


In [5]:
pandaDf.count()

accountNumber               786363
accountOpenDate             786363
acqCountry                  786363
availableMoney              786363
cardCVV                     786363
cardLast4Digits             786363
cardPresent                 786363
creditLimit                 786363
currentBalance              786363
currentExpDate              786363
customerId                  786363
dateOfLastAddressChange     786363
echoBuffer                  786363
enteredCVV                  786363
expirationDateKeyInMatch    786363
isFraud                     786363
merchantCategoryCode        786363
merchantCity                786363
merchantCountryCode         786363
merchantName                786363
merchantState               786363
merchantZip                 786363
posConditionCode            786363
posEntryMode                786363
posOnPremises               786363
recurringAuthInd            786363
transactionAmount           786363
transactionDateTime         786363
transactionType     

In [6]:
   .rdd.(pandaDf, 'Data/pickles/dataset')

ValueError: Unrecognized compression type: Data/pickles/dataset
Valid compression types are ['infer', None, 'bz2', 'gzip', 'xz', 'zip']

In [22]:
spark.sql("CREATE OR REPLACE TEMPORARY VIEW transactions USING json OPTIONS" + 
      " (path 'Data/transactions.txt')")
spark.sql("select * from transactions").show()

+-------------+---------------+----------+--------------+-------+---------------+-----------+-----------+--------------+--------------+----------+-----------------------+----------+----------+------------------------+-------+--------------------+------------+-------------------+--------------------+-------------+-----------+----------------+------------+-------------+----------------+-----------------+-------------------+--------------------+
|accountNumber|accountOpenDate|acqCountry|availableMoney|cardCVV|cardLast4Digits|cardPresent|creditLimit|currentBalance|currentExpDate|customerId|dateOfLastAddressChange|echoBuffer|enteredCVV|expirationDateKeyInMatch|isFraud|merchantCategoryCode|merchantCity|merchantCountryCode|        merchantName|merchantState|merchantZip|posConditionCode|posEntryMode|posOnPremises|recurringAuthInd|transactionAmount|transactionDateTime|     transactionType|
+-------------+---------------+----------+--------------+-------+---------------+-----------+-----------+-

In [31]:
from pyspark.sql.functions import sum, col, desc
df.groupBy("isFraud").count().show()

+-------+------+
|isFraud| count|
+-------+------+
|   true| 12417|
|  false|773946|
+-------+------+



In [32]:
df.groupBy("merchantCity").count().show()

+------------+------+
|merchantCity| count|
+------------+------+
|            |786363|
+------------+------+



In [8]:
df = pd.read_pickle('Data/pickles/dataset')

df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False


In [9]:
"Dataset Dimensions: {}x{}".format(df.shape[0], df.shape[1])

'Dataset Dimensions: 786363x29'

## Checking for nans

In [23]:
df.isnull().sum()

AttributeError: 'DataFrame' object has no attribute 'isnull'

One can only hope for clean data...

Considering our shape dimensions, I can pretty muh remove merchantCity, merchantState, merchantZip, posOnPremises, and recurringAuthInd right off the bat since there are too many missing values to do anything else while remaining accurate.
I willi keep ID related columns for now in case I am interested in looking at a specific customer's transactions.

I want to keep cardCVV and enteredCVV just in case at the moment as I'm interested to see how many of them differ from each other.

In [11]:
df.drop(['merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd'], axis=1, inplace=True)

df.isnull().sum()

df = df.reset_index(drop=True)

df

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,echoBuffer,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,2015-03-14,414,414,1803,PURCHASE,NaN,0.00,False,False,False
1,737265056,737265056,5000,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,2015-03-14,486,486,767,PURCHASE,NaN,0.00,True,False,False
2,737265056,737265056,5000,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,2015-03-14,486,486,767,PURCHASE,NaN,0.00,False,False,False
3,737265056,737265056,5000,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,2015-03-14,486,486,767,PURCHASE,NaN,0.00,False,False,False
4,830329091,830329091,5000,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,2015-08-06,885,885,3143,PURCHASE,NaN,0.00,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,732852505,732852505,50000,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,90,...,2012-08-23,936,936,3783,PURCHASE,NaN,1095.04,False,False,False
786359,732852505,732852505,50000,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,09,...,2012-08-23,939,939,3388,PURCHASE,NaN,1214.96,False,False,False
786360,732852505,732852505,50000,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,02,...,2012-08-23,936,936,3783,PURCHASE,NaN,1233.85,False,False,False
786361,732852505,732852505,50000,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,09,...,2012-08-23,939,939,3388,PURCHASE,NaN,1283.28,False,False,False


In [12]:
pd.to_pickle(df, 'Data/pickles/clean_data', protocol=4)